In [86]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from PIL import Image


In [87]:
df = pd.read_csv('/Users/user/repos/zoobot-3d/data/gz3d_and_gz_desi_matches.csv')

In [88]:
df = df[:100]  # debugging mode

In [89]:
#  '/share/nas2/walml/galaxy_zoo/decals/dr8/fits/'
df['galahad_fits_loc'] = df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.fits'


#  '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/'
df['galahad_jpg_loc'] = df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'

df['galahad_fits_loc'][0], df['galahad_jpg_loc'][0]

('593252/593252_2670.fits', '593252/593252_2670.jpg')

In [90]:
with open('data/galahad_fits_to_copy.txt', 'w') as f:
    f.write('\n'.join(df['galahad_fits_loc']))

with open('data/galahad_jpg_to_copy.txt', 'w') as f:
    f.write('\n'.join(df['galahad_jpg_loc']))

In [91]:
# rsync --files-from data/galahad_fits_to_copy.txt -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/fits data/gz_desi/fits

In [92]:
# rsync --files-from data/galahad_jpg_to_copy.txt -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/jpg data/gz_desi/jpg

In [93]:
df['local_desi_fits_loc'] = 'data/desi/fits/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.fits'
df['local_desi_jpg_loc'] = 'data/desi/jpg/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'

In [94]:
df['local_json_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/marks/').replace('.fits.gz', '.json'))

In [95]:
df['local_spiral_mask_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/masks/').replace('.fits.gz', '_spiral.jpg'))
df['local_bar_mask_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/masks/').replace('.fits.gz', '_bar.jpg'))

In [96]:
# local_gz3d_locs = glob.glob('data/gz3d/fits_gz/*.fits.gz')
# temp = pd.DataFrame(data=local_gz3d_locs, columns=['local_gz3d_fits_loc'])
# temp['mangaid'] = temp['local_gz3d_fits_loc'].apply(lambda x: os.path.basename(x).split('_')[1])

# df = pd.merge(df, temp, on='mangaid', how='left')
# # TODO can remove this soon


In [97]:
df['local_spiral_mask_loc'][0]

'data/gz3d/segmaps/masks/gz3d_1-379196_127_14728482_spiral.jpg'

In [98]:
len(df)

100

In [99]:
df = df[df['local_gz3d_fits_loc'].apply(lambda x: os.path.isfile(x))]
len(df)

100

In [100]:
df.to_csv('data/gz3d_and_gz_desi_master_catalog.csv', index=False)